WeatherPy

In [18]:
# Dependencies and Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time
import json
from scipy import stats
from scipy.stats import linregress

# Import API key
from api_keys import weather_api

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90,90)
lng_range = (-180, 180)

Generate Cities List

In [19]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat an lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=25)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=25)
lat_lngs = zip(lats,lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # If the city is unique, then add it to a cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

21

Perform API Calls

In [20]:
# Save url configuration
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# See API layout
test_query = f"{url}q={city}&appid={weather_api}&units={units}"
test_response = requests.get(test_query)
test_json_response = test_response.json()
print(json.dumps(test_json_response, indent=4))

{
    "coord": {
        "lon": 6.477,
        "lat": 50.5295
    },
    "weather": [
        {
            "id": 803,
            "main": "Clouds",
            "description": "broken clouds",
            "icon": "04d"
        }
    ],
    "base": "stations",
    "main": {
        "temp": 53.92,
        "feels_like": 53.58,
        "temp_min": 50.86,
        "temp_max": 58.42,
        "pressure": 1019,
        "humidity": 97,
        "sea_level": 1019,
        "grnd_level": 975
    },
    "visibility": 10000,
    "wind": {
        "speed": 3.76,
        "deg": 188,
        "gust": 5.08
    },
    "clouds": {
        "all": 79
    },
    "dt": 1624684563,
    "sys": {
        "type": 2,
        "id": 2006332,
        "country": "DE",
        "sunrise": 1624677818,
        "sunset": 1624737006
    },
    "timezone": 7200,
    "id": 2838744,
    "name": "Schleiden",
    "cod": 200
}


In [25]:
# Set empty lists to hold Temperature, Humidity, Cloudiness, Wind Speed, & Latitude
name = []
number = []
temp = []
humid = []
cloud = []
wind = []
lat = []

# Create starting point for record count
record_count = 0

# Print beginning text
print(f"Beginning Data Retrieval")
print(f"-------------------------")


# Loop through each city
for city in cities:

    # Create search query, make reqeust and store in json
    query_url = f"{url}q={city}&appid={weather_api}&units={units}"
    response = requests.get(query_url)
    json_response = response.json()
    record_count = record_count + 1

    # Try to grab data if available in API
    try:
        name.append(json_response["name"])
        number.append(json_response["id"])
        temp.append(json_response["main"]["temp"])
        humid.append(json_response["main"]["humidity"])
        cloud.append(json_response["clouds"]["all"])
        wind.append(json_response["wind"]["speed"])
        lat.append(json_response["coord"]["lat"])               
        print(f"Processing Record {record_count}: {name}")
    
        

    # Handle exceptions for a city that is not available in SPI
    except:
        # Append null values
        print("City not found. Skipping...")
        pass

print(f"-------------------------")
print(f"Data Retrieval Complete")
print(f"-------------------------")

Beginning Data Retrieval
-------------------------
Processing Record 1: ['Jaciara']
Processing Record 2: ['Jaciara', 'Punta Arenas']
Processing Record 3: ['Jaciara', 'Punta Arenas', 'Rikitea']
Processing Record 4: ['Jaciara', 'Punta Arenas', 'Rikitea', 'Kenai']
Processing Record 5: ['Jaciara', 'Punta Arenas', 'Rikitea', 'Kenai', 'Quatre Cocos']
Processing Record 6: ['Jaciara', 'Punta Arenas', 'Rikitea', 'Kenai', 'Quatre Cocos', 'Gravdal']
Processing Record 7: ['Jaciara', 'Punta Arenas', 'Rikitea', 'Kenai', 'Quatre Cocos', 'Gravdal', 'Olafsvik']
Processing Record 8: ['Jaciara', 'Punta Arenas', 'Rikitea', 'Kenai', 'Quatre Cocos', 'Gravdal', 'Olafsvik', 'Bluff']
Processing Record 9: ['Jaciara', 'Punta Arenas', 'Rikitea', 'Kenai', 'Quatre Cocos', 'Gravdal', 'Olafsvik', 'Bluff', 'Bredasdorp']
Processing Record 10: ['Jaciara', 'Punta Arenas', 'Rikitea', 'Kenai', 'Quatre Cocos', 'Gravdal', 'Olafsvik', 'Bluff', 'Bredasdorp', 'Martinsburg']
Processing Record 11: ['Jaciara', 'Punta Arenas', 'Rik

In [28]:
new_df = pd.DataFrame({"City": name, "City ID": number, "Temperature": temp, "Humidity": humid, "Cloudiness": cloud, "Wind Speed": wind, "Latitude": lat})
new_df.head()

,City,City ID,Temperature,Humidity,Cloudiness,Wind Speed,Latitude
0,Jaciara,3460355,67.03,49,4,2.37,-15.9653
1,Punta Arenas,3874787,33.91,86,0,12.66,-53.1500
2,Rikitea,4030556,74.39,72,22,4.41,-23.1203
3,Kenai,5866063,52.30,75,90,6.91,60.5544
4,Quatre Cocos,1106643,69.10,81,85,13.11,-20.2078
